In [ ]:
import numpy as np
from scipy.io import loadmat
import tensorflow as tf
from sklearn.model_selection import TimeSeriesSplit

# 加载MAT文件
data_m = loadmat(".\\6train_data_withCorrelation_20.mat")
train_data = data_m["train_data"]  # 2998*10的矩阵

# 提取数据
X1 = []
X2 = []
Y = []
for i in range(1440):
    d = train_data[0][i]
    d = np.transpose(d)
    x1 = d[2:][:][::-1]
    x2 = d[0:2][::-1]
    y = d[0][-1]
    X1.append(x1)
    X2.append(x2)
    Y.append(y)
data_x1 = np.array(X1)
data_x2 = np.array(X2)
data_y = np.expand_dims(np.array(Y), axis=1)

# 分配训练集和测试集
tscv = TimeSeriesSplit(n_splits=4)
list_train = []
list_test = []
for train, test in tscv.split(data_x1):
    list_train.append(train)
    list_test.append(test)
num2 = list_train[3]
num1 = list_test[3]

# 划分训练集和测试集
x1_train = data_x1[num2]
x2_train = data_x2[num2]
y_train = data_y[num2]
print('x1_train', np.shape(x1_train))
print('x2_train', np.shape(x2_train))
print('y_train', np.shape(y_train))
x1_test = data_x1[num1]
x2_test = data_x2[num1]
y_test = data_y[num1]
print('x1_test', np.shape(x1_test))
print('x2_test', np.shape(x2_test))
print('y_test', np.shape(y_test))

# 生成批次
train_queue = tf.train.slice_input_producer([x1_train, x2_train, y_train], shuffle=None)
val_queue = tf.train.slice_input_producer([x1_test, x2_test, y_test], shuffle=None)

batch_xt1, batch_xt2, batch_yt = tf.train.batch(train_queue, batch_size=1152, capacity=1440)
batch_xv1, batch_xv2, batch_yv = tf.train.batch(val_queue, batch_size=288, capacity=1440)
print('batch_xv1', np.shape(batch_xv1))
print('batch_xv2', np.shape(batch_xv2))
print('batch_yv', np.shape(batch_yv))

In [ ]:
import tensorflow as tf # TensorFlow>2.2
import numpy as np

# Encoder
enc_inp1 = tf.keras.Input(shape=(5, 55), name="inp1")  # 过去时间步的输入数据
enc_inp2 = tf.keras.Input(shape=(2, 55), name="inp2")  # 未来时间步的输入数据
print('enc_inp1', enc_inp1.shape)
print('enc_inp2', enc_inp2.shape)
print('enc_inp2[:,0,:] is the recent data, enc_inp2[:,1,:] is the future data')

# Decoder
expected_output = tf.keras.Input(shape=(1,), name="expected_output")  # 预期输出
pharse = tf.keras.Input(shape=(), dtype=tf.bool, name='training')  # 是否处于训练阶段的标志位

layers_stacked_count = 2
hidden_dim = 64
de_length = 2
output_dim = 6
alpha = 0.1  # 输出损失1的权重
alpha_w = 0.01  # L2 正则化项的权重

# Encoder cell
cells = []
for i in range(layers_stacked_count):
    cells.append(tf.keras.layers.GRUCell(hidden_dim))
cell = tf.keras.layers.StackedRNNCells(cells)

enc_outputs, enc_memory = tf.keras.layers.RNN(cell, return_sequences=True)(enc_inp1)# 编码器的输出和状态

print('enc_outputs', enc_outputs.shape)

# Decoder cell
decells = []
for i in range(layers_stacked_count):
    decells.append(tf.keras.layers.GRUCell(hidden_dim))
decell = tf.keras.layers.StackedRNNCells(decells)

dec_state = enc_memory  # 解码器初始状态为编码器的状态
dec_inp = enc_inp2[:, 1, -6:]  # 解码器初始输入为未来时间步的数据
dec_outputs = []
outs = []

w_h = tf.Variable(tf.random.normal([hidden_dim, 6]))  # 权重矩阵
b_h = tf.Variable(tf.random.normal([6]))  # 偏置向量

for time_step in range(de_length):
    if time_step > 0:
        dec_inp = out1  # 在非第一个时间步时，解码器的输入为上一个时间步的输出
    dec_output, dec_state = decell(dec_inp, dec_state)  # 解码器的输出和状态
    out1 = tf.matmul(dec_output, w_h) + b_h  # 解码器的输出
    if pharse:
        dec_inp = enc_inp2[:, time_step + 1, -6:]  # 在训练阶段，下一个时间步的输入为真实未来数据
    else:
        dec_inp = out1  # 在测试阶段，下一个时间步的输入为上一个时间步的输出
    outs.append(out1)
    dec_outputs.append(dec_output)

print('dec_state', dec_state.shape)

w_out = tf.Variable(tf.random.normal([hidden_dim, output_dim]))  # 输出层权重矩阵
b_out = tf.Variable(tf.random.normal([output_dim]))  # 输出层偏置向量

pre_output1 = outs[0]  # 预测的输出1
pre_output2 = tf.matmul(dec_outputs[-1], w_out) + b_out  # 预测的输出2

print(pre_output1.shape)
print(pre_output2.shape)

# Loss
output_loss1 = tf.sqrt(tf.reduce_mean(tf.square(enc_inp2[:, 1, -7:-1] - pre_output1)))  # 输出损失1：目标点之前到现在的点的均方根误差
output_loss2 = tf.sqrt(tf.reduce_mean(tf.square(expected_output - pre_output2)))  # 输出损失2：预测输出与目标输出的均方根误差

reg_loss = tf.reduce_sum([tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()
                          if not ("Bias" in tf_var.name or "Output_" in tf_var.name)])  # L2 正则化项

loss = alpha * output_loss1 + output_loss2 + alpha_w * reg_loss  # 总损失函数
tf.summary.scalar('loss', loss)

# Optimizer
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.999)
train_op = optimizer.minimize(loss)
merged_summary = tf.summary.merge_all()

# Evaluation metrics
acc = 1 - tf.reduce_mean(tf.sqrt(tf.square(expected_output - pre_output2)) / tf.sqrt(tf.square(expected_output)))
tf.summary.scalar('ACC', acc)

rmse = tf.sqrt(tf.reduce_mean(tf.square(pre_output2 - expected_output)))
tf.summary.scalar('RMSE', rmse)

unexplained_error = tf.reduce_sum(tf.square(expected_output - pre_output2))
total_error = tf.reduce_sum(tf.square(expected_output - tf.reduce_mean(expected_output, axis=0)))
r_square = 1. - tf.divide(unexplained_error, total_error)
tf.summary.scalar('R_square', r_square)